### 접근
- $O(N \log N)$ 까지 풀 수 있는 문제이다.
- 어떤 $A, B$ XOR 했을 때 그 값이 가장 크다는 것은 $A + B$ 가 크거나, $A \land B$ 가 가장 적어야 한다.
- 실제로 정렬해서 풀 순 없지만, 정렬 했을 때 맨 처음과 맨 마지막을 XOR 한 것이 항상 최적해일까?
  - 우선 조금만 생각해도 아닌것을 알 수 있다. 반례: 1 3 12
  - 게다가 XOR 합을 최대로 만드는 두 수는 $L[i]$ 단일이 아닌, 특정 구간을 XOR 합 때에만 나오는 수 일 수 있다.
- 비트 집합의 관점에서 생각해보면, 수열을 적당히 xor해서 가능한 가장 길고 -> True인 bit가 많아야 이득이라고 볼 수 있다.
  - 어떻게 활용해야할진 모르겠다.
- 우선 max, xor를 노드로 두는 세그트리를 만들어 보기로 했다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from dataclasses import dataclass
@dataclass
class Node :
  xor: int = 0
  max: int = 0

class ST : 
  def __init__(self, L):
    self._size = _size = 1 << (len(L) - 1).bit_length()
    self.seg = [Node() for _ in range(2 * _size)]
    self.seg[_size:_size + len(L)] = [Node(xor = v, max = v) for v in L]
    self.init()

  def merge(self, l: Node, r: Node):
    ret = Node()
    ret.xor = l.xor ^ r.xor
    ret.max = max(l.max, r.max, ret.xor)
    return ret

  def init(self):
    for i in reversed(range(self._size)) :
      L = self.seg[i * 2]
      R = self.seg[i * 2 + 1]
      self.seg[i] = self.merge(L, R)
      
  def __getitem__(self, i) -> Node:
    return self.seg[i + self._size]

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  st = ST(L)
  print(st.seg[0].max)

for _ in range(int(input())) :
  sol()

- WA를 받았다. 그도 그럴것이, 루트 노드의 값이라 함은 모든 구간을 전부 따졌을 때에만 가능하다. \
만약 최적해가 [2, N-2] 쯤에서 존재한다고 하면 루트 노드엔 그것이 반영되지 않는다는 뜻이다.\
적절히 쿼리를 해야 하는 문제인지 잘 모르겠다. 애초에 세그가 잘못됐을지도.

- $L$ 에서 가장 큰 수의 비트집합 크기가 $N$ 이라고 했을 때, 최적해의 upper bound는 $2^N - 1$ 이라는 것을 발견했다. 즉, 해당 수를 이진수로 바꿨을 때의 bitmask 보다 작다는 것이다.
- 그렇다면 $L$ 에서 비트 집합의 크기가 $N$ 인 것들을 $P$ 라고 하고, 그 좌/우 구간을 살피면서 그 구간의 xor합이 $p$ 의 complementary bit가 가장 많은 수를 찾는다는 접근은 어떨까?
  - 문제는, 구간을 어떻게 잡아야 하냐는 것이다. 구간의 크기는 최대 $N-1$ 일 수 있는데, 최적해를 만드는 complementary를 어떻게 찾아야 한다는 뜻일까?
  - 연속하는 수열을 잡아야 한다는 것을 다시 상기했더니 느낌이 왔다. 구간을 1씩 늘려나가면서 complementary를 찾는 것이다.
    - 이때의 최악케이스는 $\frac {N}{2}$ 개의 pivot이 한 칸 걸러 존재할 때, 각 pivot이 양쪽 1개씩의 구간, 즉 2개의 쿼리를 발생시킬 때일 것이다.\
    쿼리는 $O(\log N)$ 이므로, $O(N \log N)$ 이다. 할만한 풀이같다.
    - 사실 양쪽 끝이 pivot일 때 나머지 구간을 2번씩 살펴보는 경우가 최악인 것 같다. 그럼에도 $O(2N \log N) = O(N \log N)$ 이다.
  - 반례가 있다. pivot을 짝수개 통과했을 때 최적해가 되는 경우가 있다. ex) 001000, 100000, 100000, 110111

- 트라이로 XOR의 최댓값을 찾는 웰노운 유형을 배웠다.
  - 우선 있는 숫자로 trie를 만든다.
  - 숫자 $A$ 의 쿼리에 대해 트라이상에서 가능한 XOR의 최댓값을 찾는다. 그 값을 $Q$ 라고 하자
  - $Q$ 를 트라이에 추가한다.
  - 이렇게 하면 우선 예제 입력에 대한 정답은 나온다. 다만 이게 연속된 부분 수열을 고르는지는 잘 모르겠다. 우선 시도해보자.
    - 일단 만약 그게 맞다면, 연속하는 수열이 아닌 경우엔, 뒤에 나오는 수열은 아직 trie상에 존재하지 않기 때문일 것이다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MAX = 32
class Node:
  def __init__(self):
    self.data = [False, False]
    self.child = [None, None]
  
  def __setitem__(self, k, _):
    k = bool(k)
    self.data[k] = True
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k):
    k = bool(k)
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k):
    return self.data[bool(k)]

class Trie:
  def __init__(self):
    self.root = Node()

  def add(self, n):
    cur = self.root
    for i in reversed(range(MAX)) :
      cur[n & (1 << i)] = None
      cur = cur[n & (1 << i)]

  def query(self, n): # max xor
    cur = self.root
    res = 0
    for i in reversed(range(MAX)) :
      v = not n & (1 << i)
      if v in cur :
        cur = cur[v]
        res += 1 << i
      else :
        cur = cur[not v]
    return res

  def query2(self) : #maximum number can be made in trie
    cur = self.root
    res = 0
    for i in reversed(range(MAX)) :
      if 1 in cur :
        cur = cur[1]
        res += 1 << i
      else :
        cur = cur[0]
    return res

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  trie = Trie()
  for l in L:
    trie.add(l)
  
  for l in L :
    res = trie.query(l)
    trie.add(res)
  
  print(trie.query2())

for _ in range(int(input())) :
  sol()

- 진척이 있긴 했지만, 또다시 WA를 받았다. 
- 연속하는 수열을 고르지 않거나 최댓값을 제대로 찾지 못하는 것 같다.
  - 전자인것 같다. 반례) 1 2 2 2 6, 정답: 6, 출력: 7
  - 연속하는 수열을 고른다는 것은 $i$ 번째 집합을 query한다 했을 때, $i$ 뒤의 연속하는 원소에 대해서만 비교를 해야한다.\
  하지만 지금의 구현은 연속하지 않는 경우도 trie안에 있는 상태이다.
- 처음에 모든 원소를 넣기보단, 적당히 노드를 빼준다는 아이디어가 떠올랐다.
  - remove 구현이 참 곤란하다..

### 풀이
- 연속하는 부분 수열을 고른다는 것은, 수열 $L (|L|=N)$ 구간 $[a, b](a \leq b \leq N)$ 에 대해서, 가능한 모든 $a, b$의 조합을 구하는 것과 같다고 볼 수 있다.
  - 이를테면 $[1, N]$, $[2, N-5]$, $[N-10, N]$ 등의 모든 구간을 살펴보는 것이다.
  - $[1, N]$ 이나 $[1, N-5]$ 등을 구하는 과정은 비교적 쉽다. \
  하지만, 그 외의 구간에 대해서 비교할때, 경우의 수가 $\frac {N^2}{2} $ 번 나오게 된다.
- 이를 빠르게 계산하기 위해 XOR의 특성중 하나인 $X \oplus X = 0$ 인 성질을 이용할 수 있다.
  - 구간 XOR합을 담은 집합 $A = \{0, 0 \oplus L_1, 0 \oplus L_1 \oplus L_2, \cdots , 0 \oplus \cdots \oplus L_N\}$ 이라고 정의하고,\
  $a_i \in A$ 에 대해 문제의 쿼리 $Q(x) = \max(x \oplus a_1, x \oplus a_2, \cdots , x \oplus a_N)$ 라고 할 때, $Q(a_j) (1 \leq j \leq N) $ 를 수행하는 것을 고려해보자.\
  $Q(a_j) = \max(a_j \oplus 0, a_j \oplus a_1, \cdots a_j \oplus a_j, \cdots a_j \oplus a_N)$ 이 된다.\
  그런데 $a_j = 0 \oplus L_1, \cdots \oplus L_j$ 이므로, 위의 식은 다시말해\
  $Q(a_j) = \max(a_j, L_2 \oplus L_3 \oplus \cdots \oplus L_j, L_3 \oplus L_4 \oplus \cdots \oplus L_j, \cdots , 0, L_{j+1}, L_{j+1} \oplus L_{j+2},\cdots , L_{j+1} \oplus \cdots \oplus L_N)$ 가 된다.\
  즉, $Q(a_j)$ 을 수행한다는 것은 $[1 \to j, j]$ 과 $[j, j \to N]$ 의 구간 XOR합을 모두 비교한다는 것이 된다.\
  $\forall 1 \leq j \leq N$ 에 대해 $Q(a_j)$ 를 수행하면, 원했던 구간을 모두 비교할 수 있게 된다.
- $Q(x)$ 는 이진 트라이를 이용하여 $O(L = 32)$ 만에 구할 수 있다.
- 이렇게 현재까지의 부분합을 이용하여 최대 부분수열의 연산을 구하는 접근법을 생각해볼 수 있다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
  
  def __setitem__(self, k: bool, _):
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class Trie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def __repr__(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def add(self, n):
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur[v] = None
      cur = cur[v]
  
  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), trie 상에 존재하는 원소중에 n과 XOR 연산했을 때의 최대값
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = not (n >> i) & 1
      if v in cur :
        cur = cur[v]
        res += 1 << i
      else :
        cur = cur[not v]
    return res

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  trie = Trie()
  trie.add(0)

  XOR = [0]
  xor = 0
  for v in L :
    xor ^= v
    trie.add(xor)
    XOR.append(xor)
  
  ans = 0
  for xor in XOR:
    res = trie.query(xor)
    ans = max(ans, res)
  
  print(ans)

for _ in range(int(input())) :
  sol()